# Оптимизация выполнения кода, векторизация, Numba

## Задачи для совместного разбора

In [ ]:
import numpy as np
import numba
from numba import njit
import string
import numpy as np
import pandas as pd
from numba import jit, njit
import numba
from typing import Union
import string
import re
import cProfile

1. Сгенерируйте массив `A` из `N=1млн` случайных целых чисел на отрезке от 0 до 1000. Пусть `B[i] = A[i] + 100`. Посчитайте среднее значение массива `B`.

In [ ]:
A=np.random.randint(0, 1000, 1000000)
B=A+100
B.mean()

598.913076

2. Напишите функцию, которая возвращает сумму всех чисел от 0 до x-1. Примените функцию к каждому элементу массива.

In [ ]:
def fun(x):
    return(np.arange(x).sum())
np.stack(np.vectorize(fun)(B))

array([301476,  61075, 381501, ..., 109278, 454581,  45753])

In [ ]:
funs = np.vectorize(fun)
funs(B)

array([301476,  61075, 381501, ..., 109278, 454581,  45753])

3. Создайте таблицу 2млн строк и с 4 столбцами, заполненными случайными числами. Добавьте столбец `key`, которые содержит элементы из множества английских букв. Выберите из таблицы подмножество строк, для которых в столбце `key` указаны первые 5 английских букв.

In [ ]:
N = 2_000_000
df = pd.DataFrame(np.random.randn(N, 4), columns=[f"col{i}" for i in range(4)])
df["key"] = np.random.choice(list(string.ascii_letters.lower()), N, replace=True)
print(df.head(10))
df.key.iloc[[5]]='a'
df.key.iloc[[6]]='b'
df.key.iloc[[7]]='c'
df.key.iloc[[8]]='d'
df.key.iloc[[9]]='e'

df.key.iloc[[11]]='a'
df.key.iloc[[12]]='b'
df.key.iloc[[13]]='c'
df.key.iloc[[14]]='d'
df.key.iloc[[15]]='e'
print(df.head(20))

lis_ind = []
for i, v in df.key.iteritems():
  if i < 2*10**6-4:
    if df.key.iloc[i] == 'a' and df.key.iloc[i+1] == 'b' and df.key.iloc[i+2] == 'c' and df.key.iloc[i+3] == 'd' and df.key.iloc[i+4] == 'e':
      for k in range(i, i+5):
        lis_ind.append(k)
lis_ind
df.iloc[lis_ind]


       col0      col1      col2      col3 key
0 -0.294980 -0.065791  0.963887 -0.088659   x
1  0.470072 -0.527572  0.348976 -1.754257   u
2  0.568981 -0.554069  0.247030 -0.652826   r
3  0.612285 -1.388353 -0.306669  1.037067   q
4  1.198465 -0.968923  0.099562  1.178250   w
5  1.645082  0.920079 -1.205731  0.457733   b
6  0.399873  1.751997 -1.607051  0.645313   w
7 -1.101871  1.713229  1.396857  2.528065   t
8 -0.211970  0.838793  1.703144 -1.195795   t
9 -0.454446 -0.351867 -0.332602 -1.391494   j
        col0      col1      col2      col3 key
0  -0.294980 -0.065791  0.963887 -0.088659   x
1   0.470072 -0.527572  0.348976 -1.754257   u
2   0.568981 -0.554069  0.247030 -0.652826   r
3   0.612285 -1.388353 -0.306669  1.037067   q
4   1.198465 -0.968923  0.099562  1.178250   w
5   1.645082  0.920079 -1.205731  0.457733   a
6   0.399873  1.751997 -1.607051  0.645313   b
7  -1.101871  1.713229  1.396857  2.528065   c
8  -0.211970  0.838793  1.703144 -1.195795   d
9  -0.454446 -0.351867 -

C:\Users\User\anaconda3\lib\site-packages\pandas\core\indexing.py:1637: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)


,col0,col1,col2,col3,key
5,1.645082,0.920079,-1.205731,0.457733,a
6,0.399873,1.751997,-1.607051,0.645313,b
7,-1.101871,1.713229,1.396857,2.528065,c
8,-0.211970,0.838793,1.703144,-1.195795,d
9,-0.454446,-0.351867,-0.332602,-1.391494,e
11,0.176178,-1.421222,0.773634,-0.829446,a
12,0.411193,1.735516,-0.665470,0.498135,b
13,0.079437,-1.200951,1.107618,-1.543290,c
14,0.096219,1.968328,0.188939,1.454121,d
15,2.886511,-0.273639,0.043477,0.910889,e


## Лабораторная работа 8

В файлах `recipes_sample.csv` и `reviews_sample.csv` (__ЛР 2__) находится информация об рецептах блюд и отзывах на эти рецепты соответственно. Загрузите данные из файлов в виде `pd.DataFrame` с названиями `recipes` и `reviews`. Обратите внимание на корректное считывание столбца(ов) с индексами. Приведите столбцы к нужным типам.

## Измерение времени выполнения кода

Назовем полным описанием рецепта строку, полученную путем конкатенации названия и описания рецепта через пробел. Удалите строки для рецептов, которые были добавлены не в 2010 году.

Реализуйте несколько вариантов функции подсчета средней длины полного описания рецепта для рецептов, добавленных в 2010 году.

In [ ]:
reviews = pd.read_csv('reviews_sample.csv')
reviews.rename( columns={'Unnamed: 0':'index'}, inplace=True )
recipes =  pd.read_csv("recipes_sample.csv", parse_dates=['submitted'])
recipes.head()

,name,id,minutes,contributor_id,submitted,n_steps,description,n_ingredients
0,george s at the cove black bean soup,44123,90,35193,2002-10-25,NaN,an original recipe created by chef scott meska...,18.0
1,healthy for them yogurt popsicles,67664,10,91970,2003-07-26,NaN,my children and their friends ask for my homem...,NaN
2,i can t believe it s spinach,38798,30,1533,2002-08-29,NaN,"these were so go, it surprised even me.",8.0
3,italian gut busters,35173,45,22724,2002-07-27,NaN,my sister-in-law made these for us at a family...,NaN
4,love is in the air beef fondue sauces,84797,25,4470,2004-02-23,4.0,i think a fondue is a very romantic casual din...,NaN


In [ ]:
recipes.isna().sum()

name                  0
id                    0
minutes               0
contributor_id        0
submitted             0
n_steps           11190
description         623
n_ingredients      8880
dtype: int64

1\.1 С использованием метода `DataFrame.iterrows` таблицы:

    - функция принимает на вход таблицу, содержащую рецепты за 2010 год;
    
    - нахождение полного описания рецепта осуществляется внутри цикла по `iterrows` для каждой строки по отдельности.

In [ ]:
def get_mean_len_A(df: pd.DataFrame) -> float:
    k=0 
    d=0
    for _, row in df.dropna(subset=["description"]).iterrows():
        if row["submitted"].year==2010:
            d+=len(row["name"]+" "+row["description"])
            k+=1
    return(d/k)
get_mean_len_A(recipes)

265.501300390117

1\.2. С использованием метода `DataFrame.apply` таблицы:

    - функция принимает на вход таблицу, содержащую рецепты за 2010 год;
    
    - вызываете метод apply у таблицы, в качестве аргумента передаете функцию, которая возвращает полное описание для каждой строки;
    
    - считаете среднюю длину описаний, вызвав соответствующий метод серии.

In [ ]:


def process(recipes_2010):
    recipes_2010['full_description'] = recipes_2010['name'] + ' ' + recipes_2010['description']
    return recipes_2010['full_description']


def get_mean_len_B(df: pd.DataFrame) -> float:
    k=df["submitted"].dt.year == 2010
    a = df[k].apply(process, axis = 1)
    print(a)
    count = 0
    for i in range(len(a)):
        count += len(a.iloc[i])
    return count/len(a)
get_mean_len_B(recipes)


265.501300390117

1\.3. С использованием векторизованных методов серий `pd.Series`:

    - функция принимает на вход таблицу, содержащую рецепты за 2010 год;
    
    - при помощи векторизированных операций получаете столбец с полным описанием;
    
    - при помощи векторизированных операций получаете длины полного описания;
    
    - при помощи метода серий получаете среднюю длину полных описаний. 

In [ ]:
def get_mean_len_C(df: pd.DataFrame) -> float:
    k=pd.Series(df["submitted"]).dt.year==2010
    return (pd.Series(df["name"])[k]+" "+pd.Series(df["description"])[k]).str.len().mean()
get_mean_len_C(recipes)

265.501300390117

1.4 Проверьте, что результаты работы всех написанных функций корректны и совпадают. Измерьте выполнения всех написанных функций при помощи магических команд `time` и `timeit`.

In [ ]:
print(get_mean_len_A(recipes),get_mean_len_B(recipes),get_mean_len_C(recipes))


265.501300390117 265.501300390117 265.501300390117


In [ ]:
%time get_mean_len_A(recipes)

Wall time: 2.49 s


265.501300390117

In [ ]:
%timeit get_mean_len_A(recipes)

2.43 s ± 38.4 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [ ]:
%time get_mean_len_B(recipes)

Wall time: 1.11 s


265.501300390117

In [ ]:
%timeit get_mean_len_B(recipes)

1.08 s ± 33.5 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [ ]:
%time get_mean_len_C(recipes)

Wall time: 8.01 ms


265.501300390117

In [ ]:
%timeit get_mean_len_C(recipes)

7.38 ms ± 346 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


## Анализ пошагового выполнения кода 

Вам предлагается воспользоваться функцией, которая собирает статистику о том, сколько отзывов содержат то или иное слово. 

In [ ]:
def get_word_reviews_count(df):
    word_reviews = {}
    for _, row in df.dropna(subset=["review"]).iterrows():
        recipe_id, review = row["recipe_id"], row["review"]
        words = re.sub("[^A-Za-z\s]", "", review).split(" ")
        for word in words:
            if word not in word_reviews:
                word_reviews[word] = []
            word_reviews[word].append(recipe_id)

    word_reviews_count = {}
    for _, row in df.dropna(subset=["review"]).iterrows():
        review = row["review"]
        words = re.sub("[^A-Za-z\s]", "", review).split(" ")
        for word in words:
            word_reviews_count[word] = len(word_reviews[word])
    return word_reviews_count


2.1 Найдите узкие места в коде, проанализировав код функции по шагам, используя профайлер. Сохраните результаты работы профайлера в отдельную текстовую ячейку. Выпишите (словами), что в имеющемся коде реализовано неоптимально. 

In [ ]:
cProfile.run('get_word_reviews_count(reviews)')

         81617001 function calls (80856849 primitive calls) in 54.585 seconds

   Ordered by: standard name

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
        4    0.000    0.000    0.000    0.000 <__array_function__ internals>:2(can_cast)
        4    0.000    0.000    0.000    0.000 <__array_function__ internals>:2(prod)
   253370    0.241    0.000    0.322    0.000 <frozen importlib._bootstrap>:1017(_handle_fromlist)
        1    7.321    7.321   54.532   54.532 <ipython-input-19-13d7f6b79f61>:1(get_word_reviews_count)
        1    0.054    0.054   54.585   54.585 <string>:1(<module>)
        6    0.000    0.000    0.000    0.000 _asarray.py:110(asanyarray)
    40/36    0.000    0.000    0.000    0.000 _asarray.py:23(asarray)
   506751    0.190    0.000    0.190    0.000 _dtype.py:24(_kind_name)
   506751    0.183    0.000    0.184    0.000 _dtype.py:307(_name_includes_bit_suffix)
   506751    0.914    0.000    1.360    0.000 _dtype.py:321(_name_get)
  

In [ ]:
%%time
get_word_reviews_count(reviews)

Wall time: 26.7 s


{'Last': 97,
 'week': 1509,
 'whole': 5782,
 'sides': 458,
 'of': 109239,
 'frozen': 2943,
 'salmon': 1032,
 'fillet': 92,
 'was': 89617,
 'on': 36121,
 'sale': 262,
 'in': 62796,
 'my': 44644,
 'local': 572,
 'supermarket': 95,
 'so': 46848,
 'I': 288243,
 'bought': 1514,
 'tons': 163,
 'okay': 600,
 'only': 14209,
 '': 330966,
 'but': 43035,
 'total': 512,
 'weight': 216,
 'over': 9801,
 'pounds': 291,
 'This': 39555,
 'recipe': 71751,
 'is': 56519,
 'perfect': 7807,
 'for': 123162,
 'even': 8074,
 'though': 4942,
 'it': 133219,
 'calls': 525,
 'steaks': 511,
 'cut': 6848,
 'up': 16178,
 'the': 266530,
 'into': 7085,
 'individual': 314,
 'portions': 214,
 'and': 219215,
 'followed': 4880,
 'instructions': 1007,
 'exactly': 4625,
 'Im': 8330,
 'one': 17615,
 'those': 2419,
 'food': 3498,
 'combining': 78,
 'diets': 45,
 'left': 5064,
 'out': 25697,
 'white': 3690,
 'wine': 1760,
 'added': 22099,
 'just': 25271,
 'a': 166467,
 'dash': 547,
 'vinegar': 1946,
 'instead': 11644,
 'little'

2.2  Оптимизируйте функцию и добейтесь значительного (как минимум, в 5 раз) прироста в скорости выполнения. Для демонстрации результата измерьте скорость выполнения оригинальной функции и функции, написанной вами.

In [ ]:
def get_word_reviews_count2(df):
    word_reviews = {}
    word_reviews_count = {}
    df_f = df.dropna(subset=["review"])
    
    for _, row in df_f.iterrows():
        recipe_id, review = row["recipe_id"], row["review"]
        words = re.sub("[^A-Za-z\s]", "", review).split(" ")
        for word in words:
            if word not in word_reviews:
                word_reviews[word] = []
            word_reviews[word].append(recipe_id)
            word_reviews_count[word] = len(word_reviews[word])
    return word_reviews_count

cProfile.run('get_word_reviews_count2(reviews)')

         48803856 function calls (48423783 primitive calls) in 29.424 seconds

   Ordered by: standard name

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
        2    0.000    0.000    0.000    0.000 <__array_function__ internals>:2(can_cast)
        2    0.000    0.000    0.000    0.000 <__array_function__ internals>:2(prod)
   126685    0.115    0.000    0.154    0.000 <frozen importlib._bootstrap>:1017(_handle_fromlist)
        1    5.676    5.676   29.364   29.364 <ipython-input-22-f4f9462c6c91>:1(get_word_reviews_count2)
        1    0.060    0.060   29.424   29.424 <string>:1(<module>)
        3    0.000    0.000    0.000    0.000 _asarray.py:110(asanyarray)
    20/18    0.000    0.000    0.000    0.000 _asarray.py:23(asarray)
   253375    0.088    0.000    0.088    0.000 _dtype.py:24(_kind_name)
   253375    0.088    0.000    0.088    0.000 _dtype.py:307(_name_includes_bit_suffix)
   253375    0.437    0.000    0.649    0.000 _dtype.py:321(_name_get)
 

In [ ]:
%%time
get_word_reviews_count2(reviews)

Wall time: 15.6 s


{'Last': 97,
 'week': 1509,
 'whole': 5782,
 'sides': 458,
 'of': 109239,
 'frozen': 2943,
 'salmon': 1032,
 'fillet': 92,
 'was': 89617,
 'on': 36121,
 'sale': 262,
 'in': 62796,
 'my': 44644,
 'local': 572,
 'supermarket': 95,
 'so': 46848,
 'I': 288243,
 'bought': 1514,
 'tons': 163,
 'okay': 600,
 'only': 14209,
 '': 330966,
 'but': 43035,
 'total': 512,
 'weight': 216,
 'over': 9801,
 'pounds': 291,
 'This': 39555,
 'recipe': 71751,
 'is': 56519,
 'perfect': 7807,
 'for': 123162,
 'even': 8074,
 'though': 4942,
 'it': 133219,
 'calls': 525,
 'steaks': 511,
 'cut': 6848,
 'up': 16178,
 'the': 266530,
 'into': 7085,
 'individual': 314,
 'portions': 214,
 'and': 219215,
 'followed': 4880,
 'instructions': 1007,
 'exactly': 4625,
 'Im': 8330,
 'one': 17615,
 'those': 2419,
 'food': 3498,
 'combining': 78,
 'diets': 45,
 'left': 5064,
 'out': 25697,
 'white': 3690,
 'wine': 1760,
 'added': 22099,
 'just': 25271,
 'a': 166467,
 'dash': 547,
 'vinegar': 1946,
 'instead': 11644,
 'little'

## Numba

В файле `rating_predictions.json` хранятся данные о рейтингах рецептов и прогнозных значениях рейтингов для этого рецепта, полученных при помощи модели машинного обучения. 

Напишите несколько версий функции (см. [MAPE](https://en.wikipedia.org/wiki/Mean_absolute_percentage_error)) для расчета среднего абсолютного процентного отклонения значения рейтинга отзыва на рецепт от прогнозного значения рейтинга для данного рецепта. 


Замечание 1: в формуле MAPE под $A_t$ понимается рейтинг из отзыва $t$, под $F_t$ - прогнозное значения рейтинга отзыва $t$.

Замечание 2: в результате работы функций должно получиться одно число - MAPE для всего набора данных.

3\.1 Создайте два списка `A_list` и `F_list` на основе файла `rating_predictions.json`. Напишите функцию `mape_lists` без использования векторизованных операций и методов массивов `numpy` и без использования `numba` (проитерируйтесь по спискам и вычислите суммарное значение MAPE для всех элементов, а потом усредните результат).

Измерьте время выполнения данной функции на входных данных `A_list` и `F_list`. Временем, затрачиваемым на создание списков, можно пренебречь.
    

In [ ]:
j=pd.read_json('rating_predictions.json')
def mape_lists(a,f):
    k=len(a)
    n=0
    for i in range (k):
        n+=(abs((a[i]-f[i])/f[i]))
    return(n/(k)*100)
A_list=j["rating"].values# 
F_list=j["prediction"].values


In [ ]:
%%time
mape_lists(A_list,F_list)

Wall time: 347 ms


11.949451393825532

3\.2. Создайте массивы `numpy` `A_array` и `F_array` на основе списков `A_list` и `F_list`. Напишите функцию `mape_numpy` с использованием векторизованных операций и методов массивов `numpy`.

Измерьте время выполнения данной функции на входных данных `A_array` и `F_array`. Временем, затрачиваемым на создание массивов, можно пренебречь.

In [ ]:
def mape_numpy(a,f):
    def d (a,f):
        return abs((a - f) / a)
    return np.mean(np.stack(np.vectorize(d)(a,f))) * 100

In [ ]:
%%time
mape_numpy(A_list,F_list)

Wall time: 314 ms


13.32526550399145

3\.3. Создайте объекты `numba.typed.List` `A_typed` и `F_typed` на основе списков `A_list` и `F_list`. Напишите функцию `mape_numba` без использования векторизованных операций и методов массивов `numpy`, но с использованием `numba`. 

Измерьте время выполнения данной функции на входных данных `A_typed` и `F_typed`. Временем, затрачиваемым на создание объектов `numba.typed.List`, можно пренебречь.

Измерьте время выполнения данной функции на входных данных `A_array` и `F_array`.

In [ ]:
A_list=numba.typed.List(A_list)
F_list=numba.typed.List(F_list)
@njit
def mape_numba(a, f):
    n=0.0
    k=len(a)
    for i in range (k):
        n += (abs((a[i]-f[i])/f[i]))
    return(n/k*100)

In [ ]:
%%time
mape_numba(A_list,F_list)

Wall time: 393 ms


11.949451393825532

## Векторизация

Сайт-агрегатор устроил акцию: он дарит купоны на посещение ресторана тем пользователям, оставившим отзывы, идентификатор которых является _красивым числом_. Натуральное число называется _красивым_, если первая цифра числа совпадает с последней цифрой числа. 



4\.1 Напишите функцию `is_pretty`, которая для каждого идентификатора пользователя из файла определяет, получит ли он подарок. Запрещается преобразовывать идентификатор пользователя к строке. Подтвердите корректность реализации, продемонстрировав примеры.

In [ ]:
ids = reviews["recipe_id"].values

In [ ]:
def is_pretty(n: int) -> bool:
    n, digit1 = divmod(n, 10)
    digit=digit1
    while n:
        n, digit = divmod(n, 10)
    return digit==digit1
is_pretty(10),is_pretty(101)

(False, True)

4\.2 Посчитайте с помощью функции `is_pretty` количество пользователей, которые получат подарок. Выведите это количество на экран. Измерьте время расчетов для входных данных `ids`.

In [ ]:
%%time
k=0
for i in (ids):
    k+=is_pretty(i)
k

Wall time: 573 ms


11955

4\.3. При помощи `numpy` создайте векторизованную версию функции `is_pretty`. Посчитайте с помощью этой векторизованной функции количество пользователей, которые получат подарок. Выведите это количество на экран. Измерьте время расчетов для входных данных `ids`.


In [ ]:
%%time
np.sum(np.stack(np.vectorize(is_pretty)(ids)))

Wall time: 380 ms


11955

4\.4. При помощи `numba` создайте векторизованную версию функции `is_pretty`. Посчитайте с помощью этой векторизованной функции количество пользователей, которые получат подарок. Выведите это количество на экран. Измерьте время расчетов для входных данных `ids`.


In [ ]:
numba_is_pretty = numba.vectorize(['boolean(int64)'])(is_pretty)

In [ ]:
%%time
 
print(np.sum(numba_is_pretty(ids)))

11955
Wall time: 2.99 ms
